In [2]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

2023-06-15 00:27:01.956371: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from pathlib import Path
import numpy
#import np_utils

# Reads the files and folders within the MIDI folder
# and stores the names in the list
songs = []
#folder = Path('/content/drive/MyDrive/MIDI/MIDI/')
folder = Path('content/MIDI/MIDI/')
for file in folder.rglob('*.mid'):
  songs.append(file)

In [4]:
print(f'Number of songs in the dataset: {len(songs)}')

Number of songs in the dataset: 17232


In [5]:
# Get a sample of the vast list of songs

import random
#result =  random.sample([x for x in songs], 5000)
result =  random.sample([x for x in songs], 1000)

In [6]:
# Convert the files from .mid to a list of all the notes.

from music21 import converter, instrument, note, chord
notes = []
for i,file in enumerate(result):
    print(f'{i+1}: {file}')
    try:
      midi = converter.parse(file)
      notes_to_parse = None
      parts = instrument.partitionByInstrument(midi)
      if parts: # file has instrument parts
          notes_to_parse = parts.parts[0].recurse()
      else: # file has notes in a flat structure
          notes_to_parse = midi.flat.notes
      for element in notes_to_parse:
          if isinstance(element, note.Note):
              notes.append(str(element.pitch))
          elif isinstance(element, chord.Chord):
              notes.append('.'.join(str(n) for n in element.normalOrder))
    except:
      print(f'FAILED: {i+1}: {file}')

1: content/MIDI/MIDI/Aerosmith/Pink.2.mid
2: content/MIDI/MIDI/Counting_Crows/Round_Here.mid
3: content/MIDI/MIDI/The_Beatles/Come_and_Get_It.1.mid
4: content/MIDI/MIDI/Katrina_and_the_Waves/Walking_On_Sunshine.mid
5: content/MIDI/MIDI/Ricky_Martin/She_Bangs.3.mid
6: content/MIDI/MIDI/Jamiroquai/Deeper_Underground.1.mid
7: content/MIDI/MIDI/Hawkins_Erskine/Tuxedo_Junction.mid
8: content/MIDI/MIDI/Air_Supply/Two_Less_Lonely_People_in_the_World.mid
9: content/MIDI/MIDI/Nat_King_Cole/Mona_Lisa.1.mid
10: content/MIDI/MIDI/BBE/7_Days_One_Week.mid
11: content/MIDI/MIDI/Janet_Jackson/Escapade.2.mid
12: content/MIDI/MIDI/Weezer/Jamie.mid
13: content/MIDI/MIDI/The_Beatles/Only_a_Northern_Song.mid
14: content/MIDI/MIDI/Enya/Tea-House_Moon.1.mid
15: content/MIDI/MIDI/Aqua/My_Oh_My.2.mid


/home/milagros/anaconda3/envs/AI_MUSIC/lib/python3.10/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'Lene G./Ren\xe9 Dif'>; getting generic Instrument
  warnings.warn(


16: content/MIDI/MIDI/The_Ventures/Penetration.mid
17: content/MIDI/MIDI/The_Doors/The_Crystal_Ship.2.mid
18: content/MIDI/MIDI/Bette_Midler/The_Rose.2.mid
19: content/MIDI/MIDI/Creedence_Clearwater_Revival/Long_As_I_Can_See_The_Light.mid
20: content/MIDI/MIDI/Led_Zeppelin/The_Rain_Song.1.mid
21: content/MIDI/MIDI/Elvis_Presley/Girls_Girls_Girls.mid
22: content/MIDI/MIDI/Mina/Rose_su_rose.1.mid
23: content/MIDI/MIDI/Bryan_Adams/Run_To_You.5.mid
24: content/MIDI/MIDI/The_Pointer_Sisters/Jump_For_My_Love_.1.mid
25: content/MIDI/MIDI/Andre_Hazes/Ik_meen_t.2.mid
26: content/MIDI/MIDI/Jovanotti/Chissa_Se_Stai_Dormendo.1.mid
27: content/MIDI/MIDI/Elvis_Presley/Let_Me_Be_Your_Teddy_Bear.2.mid
28: content/MIDI/MIDI/Ronny_Jordan/So_What.mid
29: content/MIDI/MIDI/Power_Tools/Unchained_Melody.mid
30: content/MIDI/MIDI/Robert_Miles/Children.mid
31: content/MIDI/MIDI/M_People/Dont_Look_Any_Further.mid
32: content/MIDI/MIDI/Backstreet_Boys/As_Long_as_You_Love_Me.1.mid
33: content/MIDI/MIDI/Pooh/Buon

/home/milagros/anaconda3/envs/AI_MUSIC/lib/python3.10/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'\xc9 dez, \xe9 cem, \xe9 mil '>; getting generic Instrument
  warnings.warn(


92: content/MIDI/MIDI/Cliff_Richard/Summer_Holiday.mid
93: content/MIDI/MIDI/Peter,_Paul_and_Mary/Puff.1.mid
94: content/MIDI/MIDI/Bobby_Bare/Detroit_City.mid
95: content/MIDI/MIDI/ABBA/Two_for_the_Price_of_One.mid
96: content/MIDI/MIDI/The_Monkees/Theme_From_The_Monkees.mid
97: content/MIDI/MIDI/George_Harrison/My_Sweet_Lord.mid
98: content/MIDI/MIDI/Eiffel_65/Quelli_che_non_hanno_eta.mid
99: content/MIDI/MIDI/Pooh/Parsifal_parte_II_.1.mid
100: content/MIDI/MIDI/John_Denver/Annies_Song.mid
FAILED: 100: content/MIDI/MIDI/John_Denver/Annies_Song.mid
101: content/MIDI/MIDI/U2/Gone.mid
102: content/MIDI/MIDI/Jeff_Beck/Hi_Ho_Silver_Lining.mid
103: content/MIDI/MIDI/Emerson,_Lake_and_Palmer/Lucky_Man.1.mid
104: content/MIDI/MIDI/Zucchero/Cosi_celeste.mid
105: content/MIDI/MIDI/Stone_Temple_Pilots/Trippin_On_A_Hole_In_A_Paper_Heart.1.mid
106: content/MIDI/MIDI/Marvin_Gaye/Sexual_Healing.mid
107: content/MIDI/MIDI/Ruggeri/Ti_avro.mid
108: content/MIDI/MIDI/Sinatra/I_Get_A_Kick_Out_Of_You.mid


...

953: content/MIDI/MIDI/Battisti/Due_mondi.mid
954: content/MIDI/MIDI/Danilo_Amerio/Bisogno_damore.mid
955: content/MIDI/MIDI/Battisti/Una_donna_per_amico.mid
956: content/MIDI/MIDI/Creedence_Clearwater_Revival/Bad_Moon_Rising.1.mid
957: content/MIDI/MIDI/REO_Speedwagon/Cant_Fight_This_Feeling.mid
958: content/MIDI/MIDI/Willeke_Alberti/Ome_Jan.mid
959: content/MIDI/MIDI/Robbie_Williams/Kids.mid
960: content/MIDI/MIDI/Shaggy/It_Wasnt_Me.mid


/home/milagros/anaconda3/envs/AI_MUSIC/lib/python3.10/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=0, channel=None, data=b'ABSDRAKT \xa9'>; getting generic Instrument
  warnings.warn(


961: content/MIDI/MIDI/Reeves/Love_Is_Like_a_Heat_Wave.mid
962: content/MIDI/MIDI/Ruth_Jacott/Ik_kan_echt_zonder_jou.mid
963: content/MIDI/MIDI/Joe_Satriani/The_Mighty_Turtle_Head.mid
964: content/MIDI/MIDI/Chris_de_Burgh/Blond_Hair,_Blue_Jeans.mid
965: content/MIDI/MIDI/The_Corrs/Toss_the_Feathers.1.mid
966: content/MIDI/MIDI/Collins_Phil/Against_All_Odds.6.mid
967: content/MIDI/MIDI/The_Beatles/Slow_Down.mid
968: content/MIDI/MIDI/Depeche_Mode/Dreaming_of_Me.2.mid
969: content/MIDI/MIDI/Crash_Test_Dummies/Mmm_Mmm_Mmm.2.mid
970: content/MIDI/MIDI/Fool_s_Garden/Lemon_Tree.1.mid
971: content/MIDI/MIDI/Luniz/I_Got_5_On_It.1.mid
972: content/MIDI/MIDI/Journey/Lovin_Touchin_Squeezin.mid
973: content/MIDI/MIDI/McCartney/Bip_Bop.mid
974: content/MIDI/MIDI/Level_42/Forever_Now.mid
975: content/MIDI/MIDI/John_Elton/Rocket_Man.1.mid
976: content/MIDI/MIDI/The_Beach_Boys/Fun,_Fun,_Fun.2.mid
977: content/MIDI/MIDI/The_Police/Bring_on_the_Night.3.mid
978: content/MIDI/MIDI/Eiffel_65/Back_in_Time.m

/home/milagros/anaconda3/envs/AI_MUSIC/lib/python3.10/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent INSTRUMENT_NAME, track=4, channel=None, data=b'\x00\x00\x00\x00F\x7f-F\x00\x00\xff\x05\nround. \x00\x00\x00\x00F\x7f\x82\x0eF\x00\x84\x1c\xff\x05\nThings \x00\x00\x00\x00R\x7fZR\x00\x00\xff\x05\x08they \x00\x00\x00\x00R\x7fZR\x00\x00\xff\x05\x06do \x00\x00\x00\x00P\x7fZP\x00\x00\xff\x05\x08look \x00\x00\x00'>; getting generic Instrument
  warnings.warn(


991: content/MIDI/MIDI/Gary_Puckett/Young_Girl.mid
992: content/MIDI/MIDI/The_Smashing_Pumpkins/Mellon_Collie_and_the_Infinite_Sadness.mid
993: content/MIDI/MIDI/Carmina_Burana/O_Fortuna.mid
FAILED: 993: content/MIDI/MIDI/Carmina_Burana/O_Fortuna.mid
994: content/MIDI/MIDI/Scott_Joplin/Maple_Leaf_Rag.mid
995: content/MIDI/MIDI/Bette_Midler/The_Wind_Beneath_My_Wings.3.mid
996: content/MIDI/MIDI/The_Robert_Cray_Band/Right_Next_Door.mid
997: content/MIDI/MIDI/Booker_T._and_The_MG_s/Green_Onions.mid
998: content/MIDI/MIDI/Antonello_Venditti/Giulio_Cesare.mid
999: content/MIDI/MIDI/Amie_Comeaux/Blue.mid
1000: content/MIDI/MIDI/The_Whispers/And_The_Beat_Goes_On.mid


In [1]:
# Save notes to Drive for future usage
with open('notes/notes', 'wb') as filepath:
  pickle.dump(notes, filepath)

NameError: name 'pickle' is not defined

In [10]:
# Load notes from Drive
# with open('/content/drive/MyDrive/notes', 'rb') as filepath:
with open('notes/notes', 'rb') as filepath:
  notes = pickle.load(filepath)

In [11]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # Get all unique pitchnames
    pitchnames = sorted(set(item for item in notes))

     # Create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        # sequence_in is a sequence_length list containing sequence_length notes
        sequence_in = notes[i:i + sequence_length]
        # sequence_out is the sequence_length + 1 note that comes after all the notes in
        # sequence_in. This is so the model can read sequence_length notes before predicting
        # the next one.
        sequence_out = notes[i + sequence_length]
        # network_input is the same as sequence_in but it containes the indexes from the notes
        # because the model is only fed the indexes.
        network_input.append([note_to_int[char] for char in sequence_in])
        # network_output containes the index of the sequence_out
        network_output.append(note_to_int[sequence_out])

    # n_patters is the length of the times it was iterated 
    # for example if i = 3, then n_patterns = 3
    # because network_input is a list of lists
    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    # Reshapes it into a n_patterns by sequence_length matrix
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # OneHot encodes the network_output
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes,n_vocab)

In [13]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(Flatten())
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    #model.add(Dense(2048))
    model.add(Dense(1024))
    #model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=['accuracy'])

    return model
model = create_network(network_input, n_vocab)

NameError: name 'Flatten' is not defined

In [ ]:
import tensorflow as tf

tf.keras.utils.plot_model(
    model,
    to_file='model.png',
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False
)

In [ ]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=5, batch_size=128, callbacks=callbacks_list)

    return model

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    # Selects a random row from the network_input
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    # Random row from network_input
    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        print(note_index)
        # Reshapes pattern into a vector
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        # Standarizes pattern
        prediction_input = prediction_input / float(n_vocab)

        # Predicts the next note
        prediction = model.predict(prediction_input, verbose=0)

        # Outputs a OneHot encoded vector, so this picks the columns
        # with the highest probability
        index = numpy.argmax(prediction)
        # Maps the note to its respective index
        result = int_to_note[index]
        # Appends the note to the prediction_output
        prediction_output.append(result)

        # Adds the predicted note to the pattern
        pattern = numpy.append(pattern,index)
        # Slices the array so that it contains the predicted note
        # eliminating the first from the array, so the model can
        # have a sequence
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
from music21 import instrument, note, stream

def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')